# Mumble creates smart input transformers for making documents.

In [1]:
    from toolz.curried import *
    from textwrap import indent
    from textwrap import dedent
    from inspect import getfullargspec
    choices = list()

In [2]:
    @choices.append
    def yaml(str: lambda x: x.strip().startswith('---')):
        """locals().update(dict(
            sum((
                list(dict.items()) for dict in __import__('yaml').safe_load_all(\"\"\"%s\"\"\")
                ), [])
        ))"""

In [3]:
    @choices.append
    def graphviz(str: lambda x: x.strip().startswith('graph') or x.strip().startswith('digraph')):
        """__import__('IPython').display.display(
            __import__('graphviz').Source(
            \"\"\"%s\"\"\"
        ))"""

In [4]:
    from nbconvert.filters.markdown_mistune import IPythonRenderer, MarkdownWithMath
    class Markdown(MarkdownWithMath):
        """A mistune.Markdown object that accumulates the source code in the markdown body.
        """
        def render(Markdown, text):
            from nbconvert.filters import ipython2python
            Markdown.renderer.source = """"""
            return [super().render(text), ipython2python(Markdown.renderer.source)][-1]

    class Renderer(IPythonRenderer):
        """A mistune.Renderer to use with `aye.Markdown`."""
        def __init__(Renderer, *args, **kwargs): 
            Renderer.source = super().__init__(*args, **kwargs) or """"""

        def block_code(Renderer, str, lang=None):
            Renderer.source += '\n' + str
            return super().block_code(str, lang=lang)

In [5]:
    from IPython.core.inputtransformer import InputTransformer    
    
    class Transformer(__import__('collections').UserList, InputTransformer):
        push = __import__('collections').UserList.append
        def reset(Transformer): 
            from inspect import getfullargspec
            source, Transformer.data = '\n'.join(Transformer), []                
            source = Markdown(Renderer()).render(source)
            for callable in choices:
                spec = getfullargspec(callable)
                if spec.annotations[spec.args[0]](source):
                    source = callable.__doc__%source
                    break
            return source
        
        def register(Transformer, object):
            TRANSFORMERS += object,

In [6]:
    def load_ipython_extension(ip=None):
        from IPython import get_ipython
        global _NATIVE_TRANSFORM
        ip = ip or get_ipython()
        _NATIVE_TRANSFORM = globals().get('_NATIVE_TRANSFORM', ip.input_transformer_manager)
        ip.input_transformer_manager.logical_line_transforms = []
        ip.input_transformer_manager.physical_line_transforms = []
        ip.input_transformer_manager.python_line_transforms = [Transformer()]

    load_ipython_extension()

# This is yaml
    
    ---
    a: 10

    assert 'a' in locals()

A literate graph builder 
    
    graph graphname {
         a -- b -- c;
         b -- d;

    }